In [10]:
!brew install deno

==> Auto-updating Homebrew...
Adjust how often this is run with `$HOMEBREW_AUTO_UPDATE_SECS` or disable with
`$HOMEBREW_NO_AUTO_UPDATE=1`. Hide these hints with `$HOMEBREW_NO_ENV_HINTS=1` (see `man brew`).
==> Auto-updated Homebrew!
Updated 3 taps (supabase/tap, homebrew/core and homebrew/cask).
==> New Formulae
attempt-cli: CLI for retrying fallible commands
fjira: Fuzzy-find cli jira interface

You have 39 outdated formulae installed.

==> Fetching downloads for: deno
==> Downloading https://ghcr.io/v2/homebrew/core/deno/manifests/2.4.5
######################################################################### 100.0%
==> Fetching dependencies for deno: readline and sqlite
==> Downloading https://ghcr.io/v2/homebrew/core/readline/manifests/8.3.1
######################################################################### 100.0%
==> Fetching readline
==> Downloading https://ghcr.io/v2/homebrew/core/readline/blobs/sha256:51a9a7122
############################################################

In [2]:
%pip install dspy langchain langchain-community yfinance

Note: you may need to restart the kernel to use updated packages.


In [3]:
# DSPy configuration for Financial Analyst agent
import os
import dspy
import yfinance as yf

# Optionally load environment from .env if available
try:
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except Exception:
    pass

# Configure model provider (OpenAI-only, consistent with other agents)
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)

dspy.configure(lm=lm)

print("DSPy configured for Financial Analyst agent.")


DSPy configured for Financial Analyst agent.


In [4]:
# Yahoo Finance tools (simple, deterministic wrappers)
from typing import List, Optional, Literal, Dict, Any


def yf_get_price(ticker: str) -> str:
    """Return latest price info for a ticker as a compact string.
    Designed for tool use within ReAct.
    """
    if not ticker or not isinstance(ticker, str):
        return "Invalid ticker"
    try:
        t = yf.Ticker(ticker)
        info = t.fast_info if hasattr(t, "fast_info") else {}
        last = getattr(info, "last_price", None) if hasattr(info, "last_price") else info.get("last_price") if isinstance(info, dict) else None
        currency = getattr(info, "currency", None) if hasattr(info, "currency") else info.get("currency") if isinstance(info, dict) else None
        if last is None:
            data = t.history(period="1d")
            if data is None or data.empty:
                return f"No price data for {ticker}."
            last = float(data["Close"].iloc[-1])
        return f"{ticker} price: {last} {currency or ''}".strip()
    except Exception as e:
        return f"Error fetching price for {ticker}: {e}"


def yf_get_history(ticker: str, period: str = "5d", interval: str = "1d") -> str:
    """Return recent historical OHLCV rows in a compact, newline-separated string.
    Keep output concise for model consumption in ReAct.
    """
    if not ticker:
        return "Invalid ticker"
    try:
        data = yf.download(tickers=ticker, period=period, interval=interval, progress=False)
        if data is None or data.empty:
            return f"No history for {ticker}."
        # Keep last up to 10 rows for brevity
        tail = data.tail(10)
        lines = [
            f"{idx.date()} O={row['Open']:.2f} H={row['High']:.2f} L={row['Low']:.2f} C={row['Close']:.2f} V={int(row['Volume'])}"
            for idx, row in tail.iterrows()
        ]
        return f"History {ticker} ({period}, {interval}):\n" + "\n".join(lines)
    except Exception as e:
        return f"Error fetching history for {ticker}: {e}"


In [5]:
# ProgramOfThought: analysis signature + module
# Reference: DSPy ProgramOfThought tutorial
# https://dspy.ai/tutorials/program_of_thought/?h=programofthought#tutorial-programofthought

class FinanceAnalysis(dspy.Signature):
    """Write Python code to analyze provided financial data/questions and output the final numeric or textual result.

    Keep code minimal and safe; print only the final result.
    """
    question = dspy.InputField()
    context = dspy.InputField()
    answer = dspy.OutputField()


# Create the ProgramOfThought program for finance analysis
finance_pot = dspy.ProgramOfThought(FinanceAnalysis)


def run_finance_analysis(question: str, context: str = "") -> str:
    """Helper wrapper to run ProgramOfThought for finance tasks."""
    try:
        pred = finance_pot(question=question, context=context)
        return getattr(pred, "answer", "")
    except Exception as e:
        return f"PoT error: {e}"


In [6]:
# ReAct signature for Financial Analysis

class FinanceReActSignature(dspy.Signature):
    """
    You are a financial analyst. Use tools to fetch market data and perform deterministic analyses.

    Tools available:
    - yf_get_price(ticker: str) -> str  # latest price summary
    - yf_get_history(ticker: str, period: str = "5d", interval: str = "1d") -> str  # recent OHLCV
    - run_finance_analysis(question: str, context: str = "") -> str  # ProgramOfThought analysis

    Behavior:
    - For data retrieval, call yf_get_price / yf_get_history.
    - For calculations, comparisons, or aggregations, call run_finance_analysis with a concise question and include retrieved context.
    - Finish with a concise, well-structured answer.

    Outputs:
    - action: primary tool used (one of: yf_get_price, yf_get_history, run_finance_analysis, answer_direct)
    - tool_result: the most relevant tool output used
    - answer: final financial analysis answer
    """
    user_message: str = dspy.InputField(description="User's finance request")
    history: dspy.History = dspy.InputField(description="Conversation history")

    reasoning: str = dspy.OutputField(description="Brief plan of tool calls and why")
    action: str = dspy.OutputField(description="Chosen primary action/tool")
    tool_result: str = dspy.OutputField(description="Tool output used")
    answer: str = dspy.OutputField(description="Final answer")


In [7]:
# FinancialAnalystAgent module using ReAct + ProgramOfThought

class FinancialAnalystAgent(dspy.Module):
    def __init__(self, max_iters: int = 5):
        super().__init__()
        self.conversation_history = dspy.History(messages=[])
        self.react = dspy.ReAct(
            FinanceReActSignature,
            tools=[yf_get_price, yf_get_history, run_finance_analysis],
            max_iters=max_iters,
        )

    def forward(self, user_message: str):
        # Append user message to internal history
        self.conversation_history.messages.append({"role": "user", "content": user_message})
        # Run ReAct with internal history
        result = self.react(user_message=user_message, history=self.conversation_history)
        # Capture assistant response back to history
        answer = getattr(result, "answer", "")
        if isinstance(answer, str) and answer.strip():
            self.conversation_history.messages.append({"role": "assistant", "content": answer})
        return result


In [8]:
# Instantiate the Financial Analyst agent
finance_agent = FinancialAnalystAgent(max_iters=5)
print("FinancialAnalystAgent ready.")


FinancialAnalystAgent ready.


In [9]:
# Example: Compare 5-day returns for two tickers using ReAct tools + ProgramOfThought

# Step 1: Retrieve recent history for two tickers (e.g., AAPL and MSFT)
hist_aapl = yf_get_history("AAPL", period="5d", interval="1d")
hist_msft = yf_get_history("MSFT", period="5d", interval="1d")

print(hist_aapl.splitlines()[0])
print(hist_aapl)
print()
print(hist_msft.splitlines()[0])
print(hist_msft)

# Step 2: Ask ProgramOfThought to compute 5-day percent returns and compare
context = f"AAPL history:\n{hist_aapl}\n\nMSFT history:\n{hist_msft}"
question = (
    "Using the provided daily Close prices for AAPL and MSFT, compute each ticker's 5-day return in percent "
    "(from oldest Close to newest Close), round to two decimals, and then state which outperformed."
)

comparison_result = run_finance_analysis(question=question, context=context)
print("\nPoT comparison result:\n", comparison_result)

# Optional: show a ReAct chat example using the agent (one shot)
resp = finance_agent("Compare AAPL and MSFT 5-day performance and provide the numeric returns.")
print({
    "answer": getattr(resp, "answer", ""),
    "action": getattr(resp, "action", ""),
    "tool_result": getattr(resp, "tool_result", "")[:500],
})


/var/folders/3f/k4sgjj1s3m1g87bckb45cbhr0000gn/T/ipykernel_29164/1346585876.py:33: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=ticker, period=period, interval=interval, progress=False)
/var/folders/3f/k4sgjj1s3m1g87bckb45cbhr0000gn/T/ipykernel_29164/1346585876.py:33: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=ticker, period=period, interval=interval, progress=False)


Error fetching history for AAPL: unsupported format string passed to Series.__format__
Error fetching history for AAPL: unsupported format string passed to Series.__format__

Error fetching history for MSFT: unsupported format string passed to Series.__format__
Error fetching history for MSFT: unsupported format string passed to Series.__format__


2025/09/17 19:52:23 ERROR dspy.predict.program_of_thought: Error in code execution: Deno executable not found. Please install Deno to proceed.
Installation instructions:
> curl -fsSL https://deno.land/install.sh | sh
*or*, on macOS with Homebrew:
> brew install deno
For additional configurations: https://docs.deno.com/runtime/getting_started/installation/


KeyboardInterrupt: 